This notebook takes processed mat/pickle files and makes a dataset for prediction.
In particular, given a history and prediction horizon, this generates all viable slices of a full trajectory.  This goes through all pickle files and constructs a dataset for k-fold cross validation.

In [1]:
import glob
import numpy as np
import scipy.io as sio
import pickle
import matplotlib.pyplot as plt
import pdb
from pkl_reader import *
from datetime import datetime
import sklearn.utils as sku

import traceback
from collections import defaultdict

Using TensorFlow backend.


In [2]:
''' CONFIG: CONSTANTS FOR EXECUTION '''

now = datetime.now()
dt_string = now.strftime('%m_%d_%H:%M:%S')
dataset_name = 'dataset_%s' % dt_string
shuffle = True   # if True, shuffle snippets according to seed (for test/train split)
seed = 0;
num_folds_cv = 5 # k-fold cross validation, number of splits (-1 = only provide full set of data)

prune_start=True          # remove stationary portion of ego's trajectory at the start
prune_end=True            # remove stationary portion of ego's trajectory at the end
min_vel_thresh=0.01       # velocity threshold (m/s) above which ego is considered moving
exclude_collisions=True  # return an empty trajectory if there was a collision

Nhist=5          # number of timesteps of motion history to predict with
Npred=20         # number of timesteps of prediction horizon
Nskip=5          # "stride" for sliding window of snippet selection
dt=0.1           # discretization (s) of full ego trajectory corresponding to N* above
ego_trans = True # whether or not to represent trajectory snippets in the ego frame
                 # if False, use the global map frame for all snippets

### TODO
* [ ] optional: complete outlier removal in bag_processing (see "Additional Notes").  Need to check time indices.
* [ ] optional: include source pkl name for snippet to trial association

In [3]:
save_ext = 'pkl'
files_to_process = glob.glob('/media/data/carla_parking_data_bkcp/bags/*.%s' % save_ext)
# files_to_process = glob.glob('/media/data/carla_parking_data_bkcp/bags/*p7_t2_e9*%s' % save_ext)

print('Found %d files to read: %s' % (len(files_to_process), files_to_process))

Found 584 files to read: ['/media/data/carla_parking_data_bkcp/bags/parking_p7_t3_e0_2020-01-10-15-00-50.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p14_t3_e9_2020-01-15-11-30-49.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p13_t2_e3_2020-01-15-10-09-08.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p7_t6_e9_2020-01-15-11-14-38.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p11_t1_e8_2020-01-14-13-34-53.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p12_t6_e6_2020-01-14-16-12-25.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p11_t2_e1_2020-01-14-13-38-29.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p10_t5_e9_2020-01-13-15-07-29.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p13_t1_e8_2020-01-15-10-06-34.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p7_t4_e5_2020-01-10-15-14-43.pkl', '/media/data/carla_parking_data_bkcp/bags/parking_p11_t5_e8_2020-01-14-14-13-39.pkl', '/media/data/carla_parking_data

In [4]:
print('Processing\n')
features_combined = []
labels_combined = []
goal_snippets_combined = []
traj_index     = [0]
skipped_files  = 0
skipped_users  = defaultdict(lambda:0)

for file in files_to_process:
    print(file)
    if save_ext == 'pkl':
        res_dict = pickle.load(open(file,'rb'))
    else:
        raise NotImplemented('Invalid Extension')
       
    goals = extract_goals(res_dict)
    
    try:
        assert len(res_dict['vehicle_object_lists'][0]) == 56, 'Sam!'
        assert goals.shape[0] == 32, 'Sam Again!'
        
        ego_trajectory, start_ind, switch_ind, end_ind, goal_ind = \
             extract_full_trajectory(res_dict, goals, prune_start, prune_end, \
                                     min_vel_thresh, exclude_collisions)

        features, labels, goal_snpts = \
            get_ego_trajectory_prediction_snippets(ego_trajectory, start_ind, switch_ind, end_ind, goal_ind, \
                                           goals, Nhist, Npred, Nskip, dt, ego_frame=ego_trans)
        
            
        features_combined.extend(features)
        labels_combined.extend(labels)
        goal_snippets_combined.extend(goal_snpts)
            
        traj_index.append(len(features) + traj_index[-1])
        
    except Exception as e:
        print(file, e)
        skipped_files += 1
        traceback.print_exc()
        
        names = file.split('_')
        skipped_users[names[4]] += 1
#         pdb.set_trace()
        
print("Num of skipped files due to exception:", skipped_files)
print("Num of skipped files of each user:", skipped_users)
        
assert(len(features_combined) == len(labels_combined) == len(goal_snippets_combined))

#     # for debugging, ensure data shuffle works.
#     ind = -2
#     plt.plot(features_combined[ind][:,0], features_combined[ind][:,1], 'b')
#     plt.plot(labels_combined[ind][:,0], labels_combined[ind][:,1], 'r')
#     for g in goal_snippets_combined[ind]:
#         plt.plot(g[0], g[1], 'kx')
#     plt.show()
#     plt.plot(features_combined[ind+1][:,0], features_combined[ind+1][:,1], 'b')
#     plt.plot(labels_combined[ind+1][:,0], labels_combined[ind+1][:,1], 'r')
#     for g in goal_snippets_combined[ind+1]:
#         plt.plot(g[0], g[1], 'kx')
#     plt.show()
N_instances = len(features_combined)
dataset_dict = {}
dataset_dict['features'] = features_combined
dataset_dict['goals']  = goal_snippets_combined
dataset_dict['labels'] = labels_combined
dataset_dict['N'] = N_instances
dataset_dict['traj_idx'] = traj_index[:-1]

pickle.dump(dataset_dict, open(dataset_name + '.pkl', 'wb'))

if shuffle:
    features_combined, labels_combined, goal_snippets_combined = \
    sku.shuffle(features_combined, labels_combined, goal_snippets_combined, random_state=seed)

if num_folds_cv > 0:
    splits = (N_instances // num_folds_cv) * np.ones(num_folds_cv)
    splits[:N_instances % num_folds_cv] += 1
    
    ind_limits = np.cumsum(splits).astype(np.int)
    
    for i in range(len(ind_limits)):
        if i == 0:
            ind_start = 0
        else:
            ind_start = ind_limits[i-1]
        ind_end = ind_limits[i]
        
        dataset_kfold = {}
        dataset_kfold['features'] = features_combined[ind_start:ind_end]
        dataset_kfold['goals'] = goal_snippets_combined[ind_start:ind_end]
        dataset_kfold['labels'] = labels_combined[ind_start:ind_end]
        dataset_kfold['N'] = ind_end - ind_start
        pickle.dump(dataset_kfold, open(dataset_name + '_fold_%d.pkl' % i, 'wb'))
                
print('Finished\n')

Processing

/media/data/carla_parking_data_bkcp/bags/parking_p7_t3_e0_2020-01-10-15-00-50.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p14_t3_e9_2020-01-15-11-30-49.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p14_t3_e9_2020-01-15-11-30-49.pkl Sam!
/media/data/carla_parking_data_bkcp/bags/parking_p13_t2_e3_2020-01-15-10-09-08.pkl


Traceback (most recent call last):
  File "<ipython-input-4-d3d2725a65ba>", line 19, in <module>
    assert len(res_dict['vehicle_object_lists'][0]) == 56, 'Sam!'
AssertionError: Sam!


/media/data/carla_parking_data_bkcp/bags/parking_p7_t6_e9_2020-01-15-11-14-38.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p11_t1_e8_2020-01-14-13-34-53.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p12_t6_e6_2020-01-14-16-12-25.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p11_t2_e1_2020-01-14-13-38-29.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p10_t5_e9_2020-01-13-15-07-29.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p13_t1_e8_2020-01-15-10-06-34.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p7_t4_e5_2020-01-10-15-14-43.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p11_t5_e8_2020-01-14-14-13-39.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p6_t5_e4_2020-01-09-18-34-14.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p12_t4_e8_2020-01-14-16-03-36.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p8_t3_e7_2020-01-12-13-54-29.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p7_t6_e5_2020-01-15-11-12-46.pkl
/media/da

Traceback (most recent call last):
  File "<ipython-input-4-d3d2725a65ba>", line 24, in <module>
    min_vel_thresh, exclude_collisions)
  File "/home/mpc/carla/PythonAPI/analysis/pkl_reader.py", line 171, in extract_full_trajectory
    switch_ind = [n for n,intent in enumerate(ego_intent) if intent > -1][0]
IndexError: list index out of range


/media/data/carla_parking_data_bkcp/bags/parking_p7_t2_e9_2020-01-10-14-59-16.pkl list index out of range
/media/data/carla_parking_data_bkcp/bags/parking_p9_t6_e0_2020-01-13-12-45-08.pkl


...

/media/data/carla_parking_data_bkcp/bags/parking_p8_t2_e2_2020-01-12-13-42-40.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p7_t4_e8_2020-01-10-15-16-12.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p13_t1_e6_2020-01-15-10-06-08.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p13_t5_e3_2020-01-15-10-20-41.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p5_t6_e1_2020-01-08-14-58-28.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p7_t4_e4_2020-01-10-15-14-10.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p13_t5_e8_2020-01-15-10-21-48.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p5_t5_e0_2020-01-08-14-52-53.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p11_t2_e8_2020-01-14-13-47-56.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p12_t2_e4_2020-01-14-15-49-45.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p11_t1_e1_2020-01-14-13-29-55.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p13_t2_e8_2020-01-15-10-11-28.pkl
/media/da

Traceback (most recent call last):
  File "<ipython-input-4-d3d2725a65ba>", line 24, in <module>
    min_vel_thresh, exclude_collisions)
  File "/home/mpc/carla/PythonAPI/analysis/pkl_reader.py", line 119, in extract_full_trajectory
    raise ValueError("Collision encountered, so skipping this instance.")
ValueError: Collision encountered, so skipping this instance.


/media/data/carla_parking_data_bkcp/bags/parking_p6_t2_e9_2020-01-09-16-04-23.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p6_t3_e5_2020-01-09-16-09-50.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p12_t1_e1_2020-01-14-15-40-30.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p6_t5_e5_2020-01-09-18-34-39.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p12_t1_e8_2020-01-14-15-44-52.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p12_t1_e5_2020-01-14-15-43-09.pkl
Collisions encountered: 
time 3927.890271395
other_id 933 vehicle.seat.leon_autopilot
normal_impulse [-19.41547966003418, -11.947650909423828, 0.5201711654663086]
time 3927.956938065
other_id 933 vehicle.seat.leon_autopilot
normal_impulse [-17.381877899169922, -10.70527172088623, 0.4631898105144501]
time 3928.023604735
other_id 933 vehicle.seat.leon_autopilot
normal_impulse [-3.915961503982544, -2.411593198776245, 0.10423136502504349]
time 3928.05693807
other_id 933 vehicle.seat.leon_autopilot
normal

Traceback (most recent call last):
  File "<ipython-input-4-d3d2725a65ba>", line 24, in <module>
    min_vel_thresh, exclude_collisions)
  File "/home/mpc/carla/PythonAPI/analysis/pkl_reader.py", line 119, in extract_full_trajectory
    raise ValueError("Collision encountered, so skipping this instance.")
ValueError: Collision encountered, so skipping this instance.


/media/data/carla_parking_data_bkcp/bags/parking_p6_t6_e2_2020-01-09-16-23-50.pkl
/media/data/carla_parking_data_bkcp/bags/parking_p8_t2_e5_2020-01-12-13-45-34.pkl
Num of skipped files due to exception: 87
Num of skipped files for each user: defaultdict(<function <lambda> at 0x7fe57c047c80>, {'p14': 58, 'p7': 11, 'p9': 2, 'p13': 7, 'p11': 3, 'p8': 3, 'p10': 2, 'p12': 1})
Finished



In [5]:
N_instances

20939

In [6]:
len(glob.glob('/media/data/carla_parking_data_bkcp/bags/*p14*.%s' % save_ext))

58